In [1]:
# import library yang dibutuhkan

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import string
import re
from collections import Counter

# Import sklearn modul

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from gensim.models import Word2Vec
import pickle

#import sastrawi
!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

#tokenize
!pip install nltk
import nltk 
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords 
nltk.download('stopwords')
nltk.download('punkt')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 7.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# upload file

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Binar-Academy/data/train_preprocess.tsv', sep='\t', names =('text', 'label'))
df.head()

,text,label
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative


In [ ]:
df.shape

(11000, 2)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    11000 non-null  object
 1   label   11000 non-null  object
dtypes: object(2)
memory usage: 172.0+ KB


In [ ]:
df.duplicated().sum()

67

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)

In [ ]:
df.isna().sum()

text     0
label    0
dtype: int64

In [ ]:
df.shape

(10933, 2)

In [ ]:
# Melihat jumlah karakter (max, min, mean) sebelum normalisasi
np.max(df.text.apply(len)), np.min(df.text.apply(len)), np.mean(df.text.apply(len))

(567, 3, 189.61456141955549)

In [ ]:
# Melihat jumlah kata (max, min, mean) sebelum normalisasi
np.max(df.text.apply(lambda x: len(x.split()))), np.min(df.text.apply(lambda x: len(x.split()))), np.mean(df.text.apply(lambda x: len(x.split())))

(110, 1, 33.060550626543495)

In [ ]:
# Membagi dataframe menjadi beberapa bagian

df_a = df.loc[:1000]
df_b = df.loc[1000:2000]
df_c = df.loc[2000:3000]
df_d = df.loc[3000:4000]
df_e = df.loc[4000:5000]
df_f = df.loc[5000:6000]
df_g = df.loc[6000:7000]
df_h = df.loc[7000:8000]
df_i = df.loc[8000:9000]
df_j = df.loc[9000:10000]
df_k = df.loc[10000:]
df_k.tail()

,text,label
10928,f - demokrat dorong upaya kemandirian energi n...,neutral
10929,tidak bosan,positive
10930,enak rasa masakan nya apalagi kepiting yang me...,positive
10931,"pagi pagi di tol pasteur sudah macet parah , b...",negative
10932,meskipun sering belanja ke yogya di riau junct...,positive


In [ ]:
#import stopword
from nltk.corpus import stopwords 
nltk.download('stopwords')
nltk.download('punkt')
stopwords_indonesia = list(set(stopwords.words('indonesian')))

factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Tambahan stopword
add_stopword = ['ya', 'sembilan','belas', 'haha', 'ribu', 'puluh', 'dua', 'sih', 'nya' \
                 ]

stopwords_indonesia.extend(add_stopword)




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def text_proses(teks):
  teks = teks.lower()
  teks = re.sub("@[A-Za-z0-9_]+"," ", teks) #Menghilangkan mention
  teks = re.sub("#[A-Za-z0-9_]+"," ", teks) #Menghilangkan hashtag
  teks = re.sub(r"\\n"," ",teks) #Menghilangkan \n
  teks = teks.strip()
  teks = re.sub(r"http\S+", " ", teks) #Menghilangkan link
  teks = re.sub(r"www.\S+", " ", teks) #Menghilangkan link
  teks = re.sub("[^A-Za-z\s']"," ", teks) #Menghilangkan yang bukan huruf
  teks = re.sub("rt"," ",teks) #Menghilangkan RT
  tokens = word_tokenize(teks)
  teks = ' '.join([word for word in tokens if word not in stopwords_indonesia])
  teks = stemmer.stem(teks)
  return teks

In [ ]:
names = ['a','b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k']
data_frames = {'a': df_a, 'b': df_b, 'c': df_c, 'd': df_d, 'e': df_e, 'f' : df_f, 'g' : df_g, 'h' : df_h, 'i' : df_i, 'j' : df_j, 'k' : df_k}

for name in names:
    data_frames[name]['text_clean'] = data_frames[name]['text'].apply(lambda x: text_proses(x))
    data_frames[name].to_csv(f'{name}_processed.csv', index=False)

<ipython-input-55-74216e6a352d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frames[name]['text_clean'] = data_frames[name]['text'].apply(lambda x: text_proses(x))
<ipython-input-55-74216e6a352d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frames[name]['text_clean'] = data_frames[name]['text'].apply(lambda x: text_proses(x))
<ipython-input-55-74216e6a352d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [ ]:
file_names = ['a_processed.csv', 'b_processed.csv', 'c_processed.csv','d_processed.csv','e_processed.csv',
              'f_processed.csv','g_processed.csv','h_processed.csv','i_processed.csv','j_processed.csv',
              'k_processed.csv']
data = []
for file in file_names:
  dd = pd.read_csv(file)
  data.append(dd)

combine_df = pd.concat(data)
combine_df.shape

(10943, 3)

In [ ]:
combine_df.to_csv('clean_data.csv', index=False)

In [ ]:
combine_df.isna().sum()

text          0
label         0
text_clean    0
dtype: int64

In [ ]:
combine_df = combine_df.dropna(axis=0).reset_index(drop=True)

In [ ]:
combine_df.duplicated().sum()

10

In [ ]:
combine_df = combine_df.drop_duplicates().reset_index(drop=True)

In [ ]:
combine_df.shape

(10925, 3)

In [ ]:
# Melihat jumlah karakter (max, min, mean) setelah normalisasi
np.max(combine_df.text_clean.apply(len)), 
np.min(combine_df.text_clean.apply(len)), 
np.mean(combine_df.text_clean.apply(len))

96.45693363844394

In [ ]:
# Melihat jumlah kata (max, min, mean) sebelum normalisasi
np.max(combine_df.text_clean.apply(lambda x: len(x.split()))), \
np.min(combine_df.text_clean.apply(lambda x: len(x.split()))), \
np.mean(combine_df.text_clean.apply(lambda x: len(x.split())))

(53, 1, 15.700320366132724)

In [ ]:
from collections import Counter

# Count unique words
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count


counter = counter_word(combine_df.text_clean)

In [ ]:
len(counter)

12614

In [ ]:
counter.most_common(20)

[('makan', 6509),
 ('enak', 3794),
 ('harga', 2026),
 ('menu', 1749),
 ('bandung', 1726),
 ('banget', 1509),
 ('restoran', 1304),
 ('layan', 1273),
 ('pilih', 1103),
 ('jalan', 1027),
 ('suasana', 986),
 ('malam', 971),
 ('nyaman', 953),
 ('coba', 952),
 ('suka', 914),
 ('goreng', 908),
 ('sepe', 869),
 ('i', 863),
 ('orang', 863),
 ('ayam', 849)]

## Menggunakan teknik Bag of Words

In [22]:

df = pd.read_csv('clean_data.csv')

In [34]:
# Proses Feature extraction

data_preprocessed = df.text_clean.to_list()
count_vect = CountVectorizer()
count_vect.fit(data_preprocessed)

X_bow = count_vect.transform(data_preprocessed)
print('Feature extraction selesai')

Feature extraction selesai


In [ ]:
# Menyimpan model Bag of Word ke format pickle
pickle.dump(count_vect, open('bow_feature.p', 'wb'))

In [35]:
classes = df.label

In [36]:
# Membagi data menjadi train dan test
Xb_train, Xb_test, yb_train, yb_test = train_test_split(X_bow, classes, test_size=0.2)

In [37]:
model = MultinomialNB()
model.fit(Xb_train, yb_train)

print('Training Selesai')

Training Selesai


In [ ]:
# simpan model ke format pickle
pickle.dump(model, open('model_bow.p', 'wb')) 

In [38]:
test = model.predict(Xb_test)

print('Testing selesai')

print(classification_report(yb_test, test))

Testing selesai
              precision    recall  f1-score   support

    negative       0.77      0.76      0.77       688
     neutral       0.79      0.62      0.70       219
    positive       0.88      0.92      0.90      1278

    accuracy                           0.84      2185
   macro avg       0.81      0.77      0.79      2185
weighted avg       0.84      0.84      0.84      2185



In [39]:
# Cross Validation

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = classes

for iteration, data in enumerate(kf.split(X_bow), start=1):


    data_train = X_bow[data[0]]
    target_train = y[data[0]]

    data_test = X_bow[data[1]]
    target_test = y[data[1]]

    clf = MultinomialNB()
    clf.fit(data_train, target_train)
    preds = clf.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))
    print('='*100)

    accuracies.append(accuracy)


    

average_accuracy = np.mean(accuracies)

print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)

Training ke- 1
              precision    recall  f1-score   support

    negative       0.75      0.76      0.76       664
     neutral       0.76      0.64      0.69       200
    positive       0.89      0.91      0.90      1321

    accuracy                           0.84      2185
   macro avg       0.80      0.77      0.78      2185
weighted avg       0.83      0.84      0.83      2185

Training ke- 2
              precision    recall  f1-score   support

    negative       0.74      0.74      0.74       663
     neutral       0.85      0.64      0.73       243
    positive       0.87      0.91      0.89      1279

    accuracy                           0.83      2185
   macro avg       0.82      0.76      0.78      2185
weighted avg       0.83      0.83      0.82      2185

Training ke- 3
              precision    recall  f1-score   support

    negative       0.76      0.78      0.77       672
     neutral       0.82      0.62      0.71       237
    positive       0.88      0

In [ ]:
# test dengan real world case

original_text = ''' 
kosong
                '''

text = count_vect.transform([text_proses(original_text)])

result = model.predict(text)[0]
print('Sentiment :\n')
print(result)

Sentiment :

negative


## Mengunakan Neural Network ( MLP Classifier )

In [40]:
model_mlp = MLPClassifier(verbose=True, random_state=42)
model_mlp.fit(Xb_train, yb_train)

print('Training selesai')

Iteration 1, loss = 0.70818163
Iteration 2, loss = 0.41115938
Iteration 3, loss = 0.27077367
Iteration 4, loss = 0.19273091
Iteration 5, loss = 0.14486705
Iteration 6, loss = 0.11449351
Iteration 7, loss = 0.09393714
Iteration 8, loss = 0.07907175
Iteration 9, loss = 0.06856508
Iteration 10, loss = 0.06044726
Iteration 11, loss = 0.05446514
Iteration 12, loss = 0.04931366
Iteration 13, loss = 0.04543429
Iteration 14, loss = 0.04259392
Iteration 15, loss = 0.03981181
Iteration 16, loss = 0.03765908
Iteration 17, loss = 0.03588283
Iteration 18, loss = 0.03423049
Iteration 19, loss = 0.03282895
Iteration 20, loss = 0.03169301
Iteration 21, loss = 0.03041568
Iteration 22, loss = 0.02958654
Iteration 23, loss = 0.02860845
Iteration 24, loss = 0.02775349
Iteration 25, loss = 0.02717417
Iteration 26, loss = 0.02652356
Iteration 27, loss = 0.02578825
Iteration 28, loss = 0.02540084
Iteration 29, loss = 0.02467645
Iteration 30, loss = 0.02420176
Iteration 31, loss = 0.02369983
Iteration 32, los

In [41]:
with open('model_MLP_bow.pickle', 'wb') as f:
    pickle.dump(model_mlp, f)

In [42]:
test = model_mlp.predict(Xb_test)

print('Testing selesai')

print(classification_report(yb_test,test))

Testing selesai
              precision    recall  f1-score   support

    negative       0.72      0.74      0.73       688
     neutral       0.71      0.50      0.58       219
    positive       0.86      0.89      0.87      1278

    accuracy                           0.80      2185
   macro avg       0.76      0.71      0.73      2185
weighted avg       0.80      0.80      0.80      2185



In [62]:
# Cross Validation

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = classes

for iteration, data in enumerate(kf.split(X_bow), start=1):


    data_train = X_bow[data[0]]
    target_train = y[data[0]]

    data_test = X_bow[data[1]]
    target_test = y[data[1]]

    clf = MLPClassifier(random_state=42, early_stopping=True, hidden_layer_sizes=50)
    clf.fit(data_train, target_train)
    preds = clf.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))
    print('='*100)

    accuracies.append(accuracy)


    

average_accuracy = np.mean(accuracies)

print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)

Training ke- 1
              precision    recall  f1-score   support

    negative       0.78      0.80      0.79       664
     neutral       0.80      0.65      0.72       200
    positive       0.91      0.92      0.91      1321

    accuracy                           0.86      2185
   macro avg       0.83      0.79      0.81      2185
weighted avg       0.86      0.86      0.86      2185

Training ke- 2
              precision    recall  f1-score   support

    negative       0.73      0.79      0.76       663
     neutral       0.87      0.45      0.60       243
    positive       0.88      0.92      0.90      1279

    accuracy                           0.83      2185
   macro avg       0.83      0.72      0.75      2185
weighted avg       0.83      0.83      0.82      2185

Training ke- 3
              precision    recall  f1-score   support

    negative       0.78      0.83      0.81       672
     neutral       0.84      0.59      0.70       237
    positive       0.90      0

## ## Menggunakan teknik Tf-Idf

In [5]:
df = pd.read_csv('clean_data.csv')
df.shape

(10925, 3)

In [6]:
df.isnull().sum()

text          0
label         0
text_clean    0
dtype: int64

In [ ]:
df = df.dropna().reset_index(drop=True)

In [7]:
df.duplicated().sum()

0

In [ ]:
df= df.drop_duplicates().reset_index(drop=True)

In [ ]:
df.shape

(10925, 3)

In [ ]:
df.to_csv('clean_data.csv', index=False)

In [8]:
# Count unique words
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count
    
counter = counter_word(df.text_clean)
len(counter)

12614

In [9]:
counter.most_common(10)

[('makan', 6509),
 ('enak', 3794),
 ('harga', 2026),
 ('menu', 1749),
 ('bandung', 1726),
 ('banget', 1509),
 ('restoran', 1304),
 ('layan', 1273),
 ('pilih', 1103),
 ('jalan', 1027)]

In [44]:
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df.text_clean.to_list())
print('Feature Extraction Selesai')


Feature Extraction Selesai


In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create an instance of TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer to the text data and transform it
tfidf_matrix = vectorizer.fit_transform(df['text_clean'])

# Get the feature names (terms)
feature_names = vectorizer.get_feature_names()

# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# Concatenate the TF-IDF DataFrame with the label column
tfidf_with_labels = pd.concat([tfidf_df, df['label']], axis=1)

# Print the resulting DataFrame
print(tfidf_with_labels)


        aa  aakan  aan  aape  aas   ab  abad  abadi  abai  abak  ...  zoo  \
0      0.0    0.0  0.0   0.0  0.0  0.0   0.0    0.0   0.0   0.0  ...  0.0   
1      0.0    0.0  0.0   0.0  0.0  0.0   0.0    0.0   0.0   0.0  ...  0.0   
2      0.0    0.0  0.0   0.0  0.0  0.0   0.0    0.0   0.0   0.0  ...  0.0   
3      0.0    0.0  0.0   0.0  0.0  0.0   0.0    0.0   0.0   0.0  ...  0.0   
4      0.0    0.0  0.0   0.0  0.0  0.0   0.0    0.0   0.0   0.0  ...  0.0   
...    ...    ...  ...   ...  ...  ...   ...    ...   ...   ...  ...  ...   
10920  0.0    0.0  0.0   0.0  0.0  0.0   0.0    0.0   0.0   0.0  ...  0.0   
10921  0.0    0.0  0.0   0.0  0.0  0.0   0.0    0.0   0.0   0.0  ...  0.0   
10922  0.0    0.0  0.0   0.0  0.0  0.0   0.0    0.0   0.0   0.0  ...  0.0   
10923  0.0    0.0  0.0   0.0  0.0  0.0   0.0    0.0   0.0   0.0  ...  0.0   
10924  0.0    0.0  0.0   0.0  0.0  0.0   0.0    0.0   0.0   0.0  ...  0.0   

       zoom  zubir  zulfikri  zulkarnain  zup  zupa  zuppa  zwitsal     lab

In [ ]:
# Menyimpan model tfidf ke format pickle
pickle.dump(tfidf, open('tfidf_feature.p', 'wb'))

## Menggunakan model Naive Bayes

In [ ]:
classes = df.label

# Membagi data menjadi train dan test
X_tfidf_train, X_tfidf_test, y_tfidf_train, y_tfidf_test = train_test_split(X_tfidf, classes, test_size=0.2)

model = MultinomialNB()
model.fit(X_tfidf_train, y_tfidf_train)

print('Training Selesai')

# simpan model ke format pickle
pickle.dump(model, open('model_tfidf_naive_bayes.p', 'wb')) 

test = model.predict(X_tfidf_test)

print('Testing selesai')

print(classification_report(y_tfidf_test, test))

Training Selesai
Testing selesai
              precision    recall  f1-score   support

    negative       0.70      0.69      0.69       663
     neutral       0.96      0.22      0.36       246
    positive       0.82      0.94      0.88      1276

    accuracy                           0.79      2185
   macro avg       0.83      0.62      0.64      2185
weighted avg       0.80      0.79      0.76      2185



In [ ]:
# Cross Validation

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = classes

for iteration, data in enumerate(kf.split(X_tfidf), start=1):


    data_train = X_tfidf[data[0]]
    target_train = y[data[0]]

    data_test = X_tfidf[data[1]]
    target_test = y[data[1]]

    clf = MultinomialNB()
    clf.fit(data_train, target_train)
    preds = clf.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))
    print('='*100)

    accuracies.append(accuracy)


    

average_accuracy = np.mean(accuracies)

print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)

Training ke- 1
              precision    recall  f1-score   support

    negative       0.72      0.72      0.72       664
     neutral       0.95      0.20      0.32       200
    positive       0.84      0.94      0.89      1321

    accuracy                           0.80      2185
   macro avg       0.84      0.62      0.64      2185
weighted avg       0.81      0.80      0.78      2185

Training ke- 2
              precision    recall  f1-score   support

    negative       0.68      0.67      0.68       663
     neutral       0.96      0.18      0.30       243
    positive       0.81      0.94      0.87      1279

    accuracy                           0.77      2185
   macro avg       0.82      0.60      0.62      2185
weighted avg       0.79      0.77      0.75      2185

Training ke- 3
              precision    recall  f1-score   support

    negative       0.72      0.70      0.71       672
     neutral       0.92      0.20      0.33       237
    positive       0.82      0

In [ ]:
# test dengan real world case

original_text = ''' 
cacian makian

                '''

text = tfidf.transform([text_proses(original_text)])

result = model.predict(text)[0]
print('Sentiment :\n')
print(result)

Sentiment :

negative


## Menggunakan MLP Classifier

In [61]:
classes = df.label

# Membagi data menjadi train dan test
X_tfidf_train, X_tfidf_test, y_tfidf_train, y_tfidf_test = train_test_split(X_tfidf, classes, test_size=0.2, random_state=42)

model_mlp_tfidf = MLPClassifier(hidden_layer_sizes=50 ,random_state=42, verbose=True, early_stopping=True)
model_mlp_tfidf.fit(X_tfidf_train, y_tfidf_train)

print('Training selesai')

Iteration 1, loss = 0.96909287
Validation score: 0.605263
Iteration 2, loss = 0.74108273
Validation score: 0.763158
Iteration 3, loss = 0.55165079
Validation score: 0.812357
Iteration 4, loss = 0.42670382
Validation score: 0.837529
Iteration 5, loss = 0.34150797
Validation score: 0.858124
Iteration 6, loss = 0.28085899
Validation score: 0.869565
Iteration 7, loss = 0.23698950
Validation score: 0.869565
Iteration 8, loss = 0.20453849
Validation score: 0.868421
Iteration 9, loss = 0.17915017
Validation score: 0.864989
Iteration 10, loss = 0.15927751
Validation score: 0.861556
Iteration 11, loss = 0.14281237
Validation score: 0.860412
Iteration 12, loss = 0.12907803
Validation score: 0.859268
Iteration 13, loss = 0.11759123
Validation score: 0.855835
Iteration 14, loss = 0.10755109
Validation score: 0.858124
Iteration 15, loss = 0.09915626
Validation score: 0.854691
Iteration 16, loss = 0.09195131
Validation score: 0.854691
Iteration 17, loss = 0.08558160
Validation score: 0.855835
Valida

In [64]:
# pickle.dump(model_mlp_tfidf, open('model_mlp_tfidf.p', 'wb'))

with open('model_mlp_tfidf_85pct.pickle','wb') as f:
  pickle.dump(model_mlp_tfidf, f)

In [63]:
test = model_mlp.predict(X_tfidf_test)

print('Testing selesai')

print(classification_report(y_tfidf_test,test))

Testing selesai
              precision    recall  f1-score   support

    negative       0.77      0.79      0.78       664
     neutral       0.79      0.62      0.70       200
    positive       0.90      0.91      0.90      1321

    accuracy                           0.85      2185
   macro avg       0.82      0.78      0.79      2185
weighted avg       0.85      0.85      0.85      2185



In [65]:
# Cross Validation

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = classes

for iteration, data in enumerate(kf.split(X_tfidf), start=1):


    data_train = X_tfidf[data[0]]
    target_train = y[data[0]]

    data_test = X_tfidf[data[1]]
    target_test = y[data[1]]

    clf = MLPClassifier(hidden_layer_sizes=50 ,random_state=42, verbose=True, early_stopping=True)
    clf.fit(data_train, target_train)
    preds = clf.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))
    print('='*100)

    accuracies.append(accuracy)


    

average_accuracy = np.mean(accuracies)

print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)

Iteration 1, loss = 0.96656889
Validation score: 0.576659
Iteration 2, loss = 0.73551053
Validation score: 0.734554
Iteration 3, loss = 0.54461165
Validation score: 0.776888
Iteration 4, loss = 0.42175012
Validation score: 0.807780
Iteration 5, loss = 0.33739027
Validation score: 0.827231
Iteration 6, loss = 0.27769260
Validation score: 0.835240
Iteration 7, loss = 0.23471042
Validation score: 0.845538
Iteration 8, loss = 0.20270302
Validation score: 0.851259
Iteration 9, loss = 0.17767390
Validation score: 0.853547
Iteration 10, loss = 0.15790040
Validation score: 0.851259
Iteration 11, loss = 0.14159867
Validation score: 0.851259
Iteration 12, loss = 0.12812276
Validation score: 0.852403
Iteration 13, loss = 0.11676598
Validation score: 0.850114
Iteration 14, loss = 0.10685897
Validation score: 0.850114
Iteration 15, loss = 0.09848288
Validation score: 0.847826
Iteration 16, loss = 0.09118103
Validation score: 0.845538
Iteration 17, loss = 0.08470873
Validation score: 0.845538
Iterat

In [66]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline

# Create a pipeline with CountVectorizer and TfidfTransformer
pipeline = Pipeline([
    ('count_vectorizer', CountVectorizer()),
    ('tfidf_transformer', TfidfTransformer())
])

# Fit the pipeline to the text data and transform it
tfidf_matrix = pipeline.fit_transform(df['text_clean'])

# Get the feature names (terms)
feature_names = pipeline['count_vectorizer'].get_feature_names()

# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# Concatenate the TF-IDF DataFrame with the label column
tfidf_with_labels = pd.concat([tfidf_df, df['label']], axis=1)

# Print the resulting DataFrame
print(tfidf_with_labels)


        aa  aakan  aan  aape  aas   ab  abad  abadi  abai  abak  ...  zoo  \
0      0.0    0.0  0.0   0.0  0.0  0.0   0.0    0.0   0.0   0.0  ...  0.0   
1      0.0    0.0  0.0   0.0  0.0  0.0   0.0    0.0   0.0   0.0  ...  0.0   
2      0.0    0.0  0.0   0.0  0.0  0.0   0.0    0.0   0.0   0.0  ...  0.0   
3      0.0    0.0  0.0   0.0  0.0  0.0   0.0    0.0   0.0   0.0  ...  0.0   
4      0.0    0.0  0.0   0.0  0.0  0.0   0.0    0.0   0.0   0.0  ...  0.0   
...    ...    ...  ...   ...  ...  ...   ...    ...   ...   ...  ...  ...   
10920  0.0    0.0  0.0   0.0  0.0  0.0   0.0    0.0   0.0   0.0  ...  0.0   
10921  0.0    0.0  0.0   0.0  0.0  0.0   0.0    0.0   0.0   0.0  ...  0.0   
10922  0.0    0.0  0.0   0.0  0.0  0.0   0.0    0.0   0.0   0.0  ...  0.0   
10923  0.0    0.0  0.0   0.0  0.0  0.0   0.0    0.0   0.0   0.0  ...  0.0   
10924  0.0    0.0  0.0   0.0  0.0  0.0   0.0    0.0   0.0   0.0  ...  0.0   

       zoom  zubir  zulfikri  zulkarnain  zup  zupa  zuppa  zwitsal     lab

## Menggunakan SVC

In [ ]:
model_svc = SVC()
model_svc.fit(X_tfidf_train, y_tfidf_train)

print('Training selesai')

Training selesai


In [ ]:
pickle.dump(model_svc, open('model_tfidf_svc.p', 'wb'))

In [ ]:
test = model.predict(X_tfidf_test)

print('Testing selesai')

print(classification_report(y_tfidf_test,test))

Testing selesai
              precision    recall  f1-score   support

    negative       0.70      0.70      0.70       650
     neutral       0.95      0.17      0.29       245
    positive       0.83      0.96      0.89      1290

    accuracy                           0.79      2185
   macro avg       0.83      0.61      0.63      2185
weighted avg       0.80      0.79      0.76      2185



In [ ]:
# Cross Validation

kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
y = classes

for iteration, data in enumerate(kf.split(X_tfidf), start=1):


    data_train = X_tfidf[data[0]]
    target_train = y[data[0]]

    data_test = X_tfidf[data[1]]
    target_test = y[data[1]]

    clf = SVC()
    clf.fit(data_train, target_train)
    preds = clf.predict(data_test)

    accuracy = accuracy_score(target_test, preds)

    print('Training ke-', iteration)
    print(classification_report(target_test, preds))
    print('='*100)

    accuracies.append(accuracy)


    

average_accuracy = np.mean(accuracies)

print('\n\n\n')
print('rata-rata akurasi : ', average_accuracy)

Training ke- 1
              precision    recall  f1-score   support

    negative       0.73      0.79      0.76       664
     neutral       0.88      0.46      0.60       200
    positive       0.89      0.92      0.91      1321

    accuracy                           0.84      2185
   macro avg       0.83      0.72      0.76      2185
weighted avg       0.84      0.84      0.83      2185

Training ke- 2
              precision    recall  f1-score   support

    negative       0.69      0.78      0.73       663
     neutral       0.89      0.40      0.55       243
    positive       0.88      0.92      0.90      1279

    accuracy                           0.82      2185
   macro avg       0.82      0.70      0.73      2185
weighted avg       0.82      0.82      0.81      2185

Training ke- 3
              precision    recall  f1-score   support

    negative       0.74      0.81      0.77       672
     neutral       0.90      0.48      0.63       237
    positive       0.90      0

# Coba Menggunakan TPOT

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11938 sha256=8610c9d228694aa66845895b4edca9f1dc1aa47415bea0f3c74117b628847181
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit


In [20]:
!pip install tpot
from tpot import TPOTClassifier



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ImportError: ignored

In [42]:

# Convert text to TF-IDF features
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df.text_clean)  # Assuming text_data contains your preprocessed text inputs
y = df.label  # Assuming label_data contains your corresponding sentiment labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the TPOT classifier (using TPOT sparse)
tpot_classifier = TPOTClassifier(generations=5, population_size=20, cv=5, random_state=42,
                                 verbosity=2, config_dict='TPOT sparse')
tpot_classifier.fit(X_train, y_train)

# Evaluate the model
accuracy = tpot_classifier.score(X_test, y_test)
print("Accuracy:", accuracy)
tpot_classifier.export('tpot_pipeline.py')

Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8457665903890159

Generation 2 - Current best internal CV score: 0.8457665903890159

Generation 3 - Current best internal CV score: 0.8458810068649886

Generation 4 - Current best internal CV score: 0.8458810068649886

Generation 5 - Current best internal CV score: 0.8458810068649886

Best pipeline: LinearSVC(input_matrix, C=0.5, dual=False, loss=squared_hinge, penalty=l2, tol=0.0001)


ValueError: ignored

In [12]:
pip install numpy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.24.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.10.1 requires scikit-learn>=1.0.2, but you have scikit-learn 0.24.2 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.3 which is incompatible.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.


In [11]:

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()


In [18]:
# Convert text to TF-IDF features
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df.text_clean)  # Assuming text_data contains your preprocessed text inputs
y = le.fit_transform(df.label)  # Assuming label_data contains your corresponding sentiment labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the TPOT classifier (using TPOT sparse)
tpot_classifier = TPOTClassifier(generations=5, population_size=20, cv=5, random_state=42,
                                 verbosity=2, config_dict='TPOT sparse')
tpot_classifier.fit(X_train, y_train)

# Evaluate the model
accuracy = tpot_classifier.score(X_test, y_test)
print("Accuracy:", accuracy)
tpot_classifier.export('tpot_pipeline.py')

NameError: ignored

In [43]:
!pip install auto-sklearn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 48.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 85.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 82.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 23.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 4.8 MB/s eta 0:00:00
  Created wheel for auto-sklear

In [1]:
import autosklearn.classification
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Convert text to TF-IDF features
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df.text_clean)  # Assuming text_data contains your preprocessed text inputs
y = df.label  # Assuming label_data contains your corresponding sentiment labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the auto-sklearn classifier
auto_classifier = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=3600,  # Maximum time in seconds for auto-sklearn to search for models
    per_run_time_limit=300,  # Maximum time in seconds for each individual model training
    ensemble_size=1,  # Number of models to include in the final ensemble
    ensemble_nbest=1,  # Number of best models to include in the final ensemble
    seed=42,  # Random seed for reproducibility
)
auto_classifier.fit(X_train, y_train)

# Evaluate the model
accuracy = auto_classifier.score(X_test, y_test)
print("Accuracy:", accuracy)


ImportError: ignored

In [ ]:
# Save the best model
best_model = auto_classifier.get_models_with_weights()[0][1]
joblib.dump(best_model, 'best_model.pkl')